In [ ]:
vcoil.ramp_to(0,steps=20)

In [ ]:
vcoil.get_current()

In [ ]:
qubit.update_instruments()
qubit.qubit_mw_src.set_status(0)
load_awg.update_sequence([qubit.exc_T*.9],gwf.square,qubit)
mspec.spec_stop()
mspec.set_segments(1)
mspec.set_blocks(1)

In [ ]:
mspec.set_averages(5e3)

In [ ]:
m.set_x_parameters(qubit.fr+arange(-0.02e9,0.02e9,0.5e6), 'freq', lambda x: readout.set_tone_freq([x]) and readout.set_LO(x+30e6))

In [ ]:
qubit.qubit_mw_src.set_status(0)
m.comment = 'pseudo vna scan'
m.dirname = 'find_fr'
m.measure_1D()

In [ ]:
data, fn, urls = dr.load_data(None,['freq','pha'])

In [ ]:
freq, pha = data
pha2 = np.unwrap(pha)
pha3 = pha2 - np.linspace(pha2[0],pha2[-1],len(pha2))
plt.plot(freq,pha3)

In [ ]:
pha_gr = np.gradient(pha3)
fr = dr.fit_data('dat_import',data_c=1,data=np.array([freq,pha_gr]),nfile=fn)[0][0]*1e9
print fr

In [ ]:
qubit.fr = fr

In [ ]:
qubit.fr

In [ ]:
readout.set_LO(qubit.fr+qubit.readout_iq_frequency)   #set LO higher and use mixer as a down-converter
readout.set_tone_freq([qubit.fr])
readout.update()

In [ ]:
qubit.save()

In [ ]:
qubit.f01 = 7e9

In [ ]:
qubit.mw_power = 10
qubit.qubit_mw_src.set_power(qubit.mw_power)
#qubit.update_instruments()

In [ ]:
#qubit.update_instruments()
qubit.qubit_mw_src.set_status(0)
load_awg.update_sequence([qubit.exc_T*.9],gwf.square,qubit)
mspec.spec_stop()
mspec.set_segments(1)
mspec.set_blocks(1)

In [ ]:
mspec.set_averages(1e4)
m.set_x_parameters(qubit.f01 + arange(-200e6, 200e6, 1e6), 'f (Hz)', qubit.qubit_mw_src.set_frequency)
#m.set_x_parameters(qubit.f01 + arange(-500e6, 500e6, 1e6), 'freq', qubit.qubit_mw_src.set_frequency)
#m.set_x_parameters(arange(5.9e9, 7e9, 1e6), 'freq', qubit.qubit_mw_src.set_frequency)
m.comment = 'homodyne'
m.dirname = 'find_f01'

In [ ]:
qubit.qubit_mw_src.set_status(1)
m.measure_1D()
qubit.qubit_mw_src.set_status(0)

In [ ]:
fq = dr.fit_data(None,ps=[6.97,0.01])[0][0]*1e9

In [ ]:
qubit.f01=fq
qubit.f01

In [ ]:
iq = qkit.instruments.create('iq','IQ_Mixer',sample=qubit,mixer_name='IQ4509')
qkit.flow.sleep(3)

In [ ]:
iq.set_sample(qubit)

In [ ]:
qubit.update_instruments()

If you have problems with the AWG, e.g. it does not respond with the right values, there is an output in the buffer and you get the response to a previous set command. 

Execute fastawg.read_queue() until there is no response any longer.

In [ ]:
fastawg.read_queue()

In [ ]:
load_awg.update_sequence([qubit.exc_T*.9],gwf.square,qubit,iq)
mspec.set_segments(1)
mspec.set_blocks(1)

In [ ]:
mspec.set_averages(1e4)
m.set_x_parameters(qubit.f01 + arange(-80e6, 80e6, 2e6), 'f (Hz)', lambda x: qubit.qubit_mw_src.set_frequency(x-qubit.iq_frequency))
m.comment = 'heterodyne'
m.dirname = 'f01_hetero'
qubit.qubit_mw_src.set_status(1)
m.measure_1D()
qubit.qubit_mw_src.set_status(0)

In [ ]:
f = dr.fit_data(None,ps=[qubit.f01*1e-9,None,None,None])[0][0]*1e9

In [ ]:
qubit.f01=f

In [ ]:
qubit.f01

In [ ]:
qubit.update_instruments()

In [ ]:
ts = np.linspace(0, 0.5, 100)   #in us
print '%d Pulses' %(len(ts))

In [ ]:
mspec.set_averages(2e3)
load_awg.update_sequence(ts*1e-6,gwf.square,qubit,iq)   #load Rabi sequence
mspec.spec_stop()
mspec.set_segments(len(ts))
mspec.set_blocks(1)

In [ ]:
m.time_data = False

In [ ]:
m.set_x_parameters(ts, 'pulse length (us)', None)
qubit.qubit_mw_src.set_status(1)
m.comment = 'find pi pulse heterodynely'
m.dirname = 'rabi_hetero'
qubit.qubit_mw_src.set_status(1)
m.measure_1D_AWG(iterations = 20)

In [ ]:
qubit.set_times(1.0e-6/dr.fit_data(None, fit_function='damped_sine')[0][0]/2)

In [ ]:
qubit.get_tpi()

In [ ]:
qubit.exc_T

In [ ]:
ts = np.linspace(0, 10, 50)   #in us
print '%d Pulses' %(len(ts))
#qubit.qubit_mw_src.set_status(0)

In [ ]:
load_awg.update_sequence(ts*1e-6, gwf.t1, qubit, iq)

In [ ]:
mspec.set_averages(2e3)
mspec.set_segments(len(ts))
mspec.set_blocks(1)
qubit.qubit_mw_src.set_status(1)

In [ ]:
m.set_x_parameters(ts, 'delay (us)', None)
m.comment = 'T1 hetero 10us window %smA, %.3gmK, fq = %s'%(str(vcoil.get_current()),tip.get_T()*1e3,str(qubit.f01))
m.dirname = 'T1_hetero_10us'

In [ ]:
qubit.qubit_mw_src.set_status(1)
m.measure_1D_AWG(iterations = 20)
qubit.qubit_mw_src.set_status(0)

In [ ]:
dr.fit_data(None, fit_function='exp')

In [ ]:
qubit.save()

In [ ]:
qubit.f01 = qubit.f01 + 3e6   #slight detuning
qubit.update_instruments()

In [ ]:
ts = np.linspace(0, 5, 150)   #in us
print '%d Pulses' %(len(ts))

In [ ]:
load_awg.update_sequence(ts*1e-6, gwf.ramsey, qubit, iq)

In [ ]:
mspec.set_averages(1e3)
mspec.set_segments(len(ts))
mspec.set_blocks(1)

In [ ]:
m.set_x_parameters(ts, 'delay (us)', None)
m.comment = 'Ramsey 5us '+str(qubit.current)
m.dirname = m.comment.replace(' ','_')
qubit.qubit_mw_src.set_status(1)
m.measure_1D_AWG(iterations = 150)
qubit.qubit_mw_src.set_status(0)

In [ ]:
dr.fit_data(None, fit_function='damped_sine')

In [ ]:
dr.PARAMS[dr.DAMPED_SINE]